<a href="https://colab.research.google.com/github/nbht98/attack_cat_or_dog_classification/blob/master/dp_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/tensorflow/privacy

Cloning into 'privacy'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 1773 (delta 5), reused 9 (delta 3), pack-reused 1755
Receiving objects: 100% (1773/1773), 593.80 KiB | 852.00 KiB/s, done.
Resolving deltas: 100% (1219/1219), done.


In [ ]:
%cd privacy
!pip install -e .

/content/privacy
Obtaining file:///content/privacy
     |████████████████████████████████| 296kB 2.8MB/s 
  Found existing installation: tensorflow-privacy 0.2.2
    Uninstalling tensorflow-privacy-0.2.2:
      Successfully uninstalled tensorflow-privacy-0.2.2
  Running setup.py develop for tensorflow-privacy


In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 1.x
except Exception:
  pass

import tensorflow as tf

import numpy as np

tf.compat.v1.logging.set_verbosity(tf.logging.ERROR)

TensorFlow 1.x selected.


In [ ]:
!pip install tensorflow_privacy

from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
from tensorflow_privacy.privacy.optimizers.dp_optimizer import DPGradientDescentGaussianOptimizer

In [ ]:
def get_data():
    """Prepare CIFAR10 data."""
    train, test = tf.keras.datasets.mnist.load_data()
    train_data, train_labels = train
    test_data, test_labels = test

    train_data = np.array(train_data, dtype=np.float32) / 255
    test_data = np.array(test_data, dtype=np.float32) / 255

    train_data = train_data.reshape(train_data.shape[0], 28, 28, 1)
    test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

    train_labels = np.array(train_labels, dtype=np.int32)
    test_labels = np.array(test_labels, dtype=np.int32)

    train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
    test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

    assert train_data.min() == 0.
    assert train_data.max() == 1.
    assert test_data.min() == 0.
    assert test_data.max() == 1.
    return (train_data, train_labels), (test_data, test_labels)
# Get MNIST dataset
(X_train, y_train), (X_test, y_test) = get_data()
    

In [ ]:
epochs = 15
batch_size = 250

In [ ]:
l2_norm_clip = 1.5
noise_multiplier = 1.3
num_microbatches = 250
learning_rate = 0.25

if batch_size % num_microbatches != 0:
  raise ValueError('Batch size should be an integer multiple of the number of microbatches')

In [ ]:
def target_model_fn():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8,
                            strides=2,
                            padding='same',
                            activation='relu',
                            input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Conv2D(32, 4,
                            strides=2,
                            padding='valid',
                            activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  optimizer = DPGradientDescentGaussianOptimizer(
      l2_norm_clip=l2_norm_clip,
      noise_multiplier=noise_multiplier,
      num_microbatches=num_microbatches,
      learning_rate=learning_rate)

  loss = tf.keras.losses.CategoricalCrossentropy(
      from_logits=True, reduction=tf.losses.Reduction.NONE)
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
  return model

In [ ]:
target_model = target_model_fn()
target_model.fit(X_train, y_train,
          epochs=epochs,
          validation_data=(X_test, y_test),
          batch_size=batch_size)
# target_model.load_weights("weights.h5")


Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.9476 - acc: 0.5913 - val_loss: 1.6578 - val_acc: 0.8323
Epoch 2/15
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.6114 - acc: 0.8698 - val_loss: 1.5675 - val_acc: 0.9031
Epoch 3/15
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5677 - acc: 0.9005 - val_loss: 1.5513 - val_acc: 0.9164
Epoch 4/15
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5518 - acc: 0.9150 - val_loss: 1.5370 - val_acc: 0.9274
Epoch 5/15
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5413 - acc: 0.9236 - val_loss: 1.5303 - val_acc: 0.9354
Epoch 6/15
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5355 - acc: 0.9288 - val_loss: 1.5290 - val_acc: 0.9356
Epoch 7/15
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5328 - acc: 0.9312 - val_loss: 1.

In [ ]:
from google.colab import files
target_model.save('weights1.h5')
files.download('weights1.h5')

In [ ]:
!pip3 install mia

  Created wheel for mia: filename=mia-0.1.2-cp36-none-any.whl size=11079 sha256=352756313b8267d06001c6ec4e59bcf642cf5cd66afe25319bba4f9ae613e4ff
  Stored in directory: /root/.cache/pip/wheels/e8/83/e4/baae7782aa0d2e45af485d25a7994bab3f76428e483252ce82
Successfully built mia


In [ ]:
# Import library 
import numpy as np

from absl import app
from absl import flags

import tensorflow as tf
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

from mia.estimators import ShadowModelBundle, AttackModelBundle, prepare_attack_data


In [ ]:
NUM_CLASSES = 10
SHADOW_DATASET_SIZE = 4000
ATTACK_TEST_DATASET_SIZE = 4000


TARGET_EPOCHS = 12 # Number of epochs to train target and shadow models
ATTACK_EPOCHS = 12 # Number of epochs to train attack models.
NUM_SHADOWS = 3 # Number of epochs to train attack models.

In [ ]:
def attack_model_fn():
    """Attack model that takes target model predictions and predicts membership.
    Following the original paper, this attack model is specific to the class of the input.
    AttachModelBundle creates multiple instances of this model for each class.
    """
    model = tf.keras.models.Sequential()

    model.add(layers.Dense(128, activation="relu", input_shape=(NUM_CLASSES,)))

    model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
    model.add(layers.Dense(64, activation="relu"))

    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile("adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model


In [ ]:
# Train the shadow models.
smb = ShadowModelBundle(
    target_model_fn,
    shadow_dataset_size=SHADOW_DATASET_SIZE,
    num_models=NUM_SHADOWS,
)

# We assume that attacker's data were not seen in target's training.
attacker_X_train, attacker_X_test, attacker_y_train, attacker_y_test = train_test_split(
    X_test, y_test, test_size=0.1
)
print(attacker_X_train.shape, attacker_X_test.shape)

print("Training the shadow models...")
X_shadow, y_shadow = smb.fit_transform(
    attacker_X_train,
    attacker_y_train,
    fit_kwargs=dict(
        epochs=TARGET_EPOCHS,
        verbose=True,
        validation_data=(attacker_X_test, attacker_y_test),
        batch_size=250
    ),
)

(9000, 28, 28, 1) (1000, 28, 28, 1)
Training the shadow models...
Train on 4000 samples, validate on 1000 samples
Epoch 1/12
4000/4000 [==============================] - 5s 1ms/sample - loss: 2.3015 - acc: 0.1107 - val_loss: 2.2988 - val_acc: 0.1010
Epoch 2/12
4000/4000 [==============================] - 5s 1ms/sample - loss: 2.2931 - acc: 0.1382 - val_loss: 2.2865 - val_acc: 0.1930
Epoch 3/12
4000/4000 [==============================] - 5s 1ms/sample - loss: 2.2746 - acc: 0.2368 - val_loss: 2.2607 - val_acc: 0.2740
Epoch 4/12
4000/4000 [==============================] - 5s 1ms/sample - loss: 2.2348 - acc: 0.4072 - val_loss: 2.1945 - val_acc: 0.5960
Epoch 5/12
4000/4000 [==============================] - 5s 1ms/sample - loss: 2.1423 - acc: 0.6118 - val_loss: 2.0721 - val_acc: 0.6740
Epoch 6/12
4000/4000 [==============================] - 5s 1ms/sample - loss: 2.0097 - acc: 0.6618 - val_loss: 1.9334 - val_acc: 0.6570
Epoch 7/12
4000/4000 [==============================] - 5s 1ms/sample 

In [ ]:
  amb = AttackModelBundle(attack_model_fn, num_classes=10)

  # Fit the attack models.
  print("Training the attack models...")
  amb.fit(
      X_shadow, y_shadow, fit_kwargs=dict(epochs=15, verbose=True)
  )

Training the attack models...
Train on 2342 samples
Epoch 1/15
2342/2342 [==============================] - 0s 182us/sample - loss: 0.6942 - acc: 0.5077
Epoch 2/15
2342/2342 [==============================] - 0s 86us/sample - loss: 0.6943 - acc: 0.4923
Epoch 3/15
2342/2342 [==============================] - 0s 88us/sample - loss: 0.6946 - acc: 0.5013
Epoch 4/15
2342/2342 [==============================] - 0s 86us/sample - loss: 0.6938 - acc: 0.5013
Epoch 5/15
2342/2342 [==============================] - 0s 93us/sample - loss: 0.6932 - acc: 0.5107
Epoch 6/15
2342/2342 [==============================] - 0s 95us/sample - loss: 0.6934 - acc: 0.5021
Epoch 7/15
2342/2342 [==============================] - 0s 90us/sample - loss: 0.6936 - acc: 0.4991
Epoch 8/15
2342/2342 [==============================] - 0s 87us/sample - loss: 0.6929 - acc: 0.5056
Epoch 9/15
2342/2342 [==============================] - 0s 91us/sample - loss: 0.6932 - acc: 0.5111
Epoch 10/15
2342/2342 [========================

In [ ]:
    # Test the success of the attack.

    # Prepare examples that were in the training, and out of the training.
    data_in = X_train[:4000], y_train[:4000]
    data_out = X_test[:4000], y_test[:4000]

    # Compile them into the expected format for the AttackModelBundle.
    attack_test_data, real_membership_labels = prepare_attack_data(
        target_model, data_in, data_out
    )

    # Compute the attack accuracy.
    attack_guesses = amb.predict(attack_test_data)
    attack_accuracy = np.mean(attack_guesses == real_membership_labels)

    print(attack_accuracy)

0.506625
